In [16]:
let parseLine (line:string) =
    line.Split(' ', StringSplitOptions.RemoveEmptyEntries)
    |> Seq.map int
    |> List.ofSeq

let nextRow row =
    row 
    |> List.pairwise
    |> List.map (fun (a,b) -> b - a)

let simplifyRow getNext row =
    let rest =
        row 
        |> List.unfold (
            fun r -> 
                if r |> List.forall ((=) 0) then 
                    None
                else
                    let next = getNext r
                    Some (next, next)
        )
    row::rest

let extrapolatedRow rowBelow row = // below here means like the inverted pyramid in the description, the 'next' row.
    let left = row |> List.last
    let below = rowBelow |> List.last
    row @ [left + below]

let extrapolateAll extrapolate rows =
    match rows |> List.rev with
    | [] -> failwith "empty list"
    | last::preceding ->
        preceding 
        |> List.scan extrapolate  (last@[0])

let extrapolatedValue extrapolatedRows =
    extrapolatedRows
    |> List.last
    |> List.last


In [17]:
#r "nuget: FsUnit.xUnit"

open FsUnitTyped

let test1 =
    "0   3   6   9  12  15"
    |> parseLine
    |> simplifyRow nextRow

test1 |> shouldEqual [ [0; 3; 6; 9; 12; 15]; [3; 3; 3; 3; 3]; [0; 0; 0; 0] ]

printfn "%A" test1

let extrapolated = extrapolateAll extrapolatedRow test1
printfn "%A" extrapolated

extrapolated |> extrapolatedValue |> shouldEqual 18

Installed Packages FsUnit.xUnit, 5.6.1

[[0; 3; 6; 9; 12; 15]; [3; 3; 3; 3; 3]; [0; 0; 0; 0]]
[[0; 0; 0; 0; 0]; [3; 3; 3; 3; 3; 3]; [0; 3; 6; 9; 12; 15; 18]]


In [19]:
open System.IO

let input = File.ReadAllLines "input_09.txt"

let result1 =
    input 
    |> Array.sumBy (parseLine >> simplifyRow nextRow >> extrapolateAll extrapolatedRow >> extrapolatedValue)

printfn "Result 1 : %d" result1
result1 |> shouldEqual 1934898178 // In case I break it

Result 1 : 1934898178
